In [1]:
import cv2 as cv
import numpy as np

In [4]:
def background_subtraction(input_video_path, output_video_path, back_sub_algo):
    if back_sub_algo == 'MOG2':
        backSub = cv.createBackgroundSubtractorMOG2()
    else:
        backSub = cv.createBackgroundSubtractorKNN()
    capture = cv.VideoCapture(input_video_path)
    if not capture.isOpened():
        print('Unable to open: ' + input_video_path)
        exit(0)
    frame_size  = (int(capture.get(cv.CAP_PROP_FRAME_WIDTH)), int(capture.get(cv.CAP_PROP_FRAME_HEIGHT)))
    frame_rate = capture.get(cv.CAP_PROP_FPS)
    fourcc = cv.VideoWriter_fourcc(*"MJPG")
    output = cv.VideoWriter(output_video_path, fourcc, frame_rate, frame_size)
    i = 0
    # Set oparams for detecting ball object
    params = cv.SimpleBlobDetector_Params()
    # Filter by area
    params.filterByArea = True
    params.minArea = 250
    params.maxArea = 1000
    # Filter by circularity
    params.filterByCircularity = True
    params.minCircularity = 0.60
    detector = cv.SimpleBlobDetector_create(params)
    start_frame = 0
    frames = 500
    capture.set(cv.CAP_PROP_POS_FRAMES, start_frame)
    last_pos = None
    while i < frames:
        if i % 10 == 0:
            print(f"{i}/{frames}")
        ret, frame = capture.read()
        if frame is None:
            break
        fgMask = backSub.apply(frame)

        kernel = np.ones((6, 6),np.uint8)
        mask = cv.morphologyEx(fgMask, cv.MORPH_OPEN, kernel)
        thresh = 127
        mask = cv.threshold(mask, thresh, 255, cv.THRESH_BINARY)[1]
        inv_mask = 255-mask
        masked_frame = cv.bitwise_and(frame, frame, mask=mask)
        blobs = detector.detect(inv_mask)
        # Filter by colour and position
        # Find average pixel color in masked image
        ball_coord = None
        for blob in blobs:
            center, radius = np.array(blob.pt, np.int32), int(blob.size/2)
            circle_mask = cv.circle(np.zeros(np.flip(frame_size), np.int8), center, radius, 255, -1)
            new_mask = cv.bitwise_and(frame, frame, mask=circle_mask)
            ball = cv.bitwise_and(new_mask, new_mask, mask=mask)
            pixels = np.sum(ball > 0)
            avg_color = ball.sum(axis=(0,1))/pixels
            if (avg_color > 60).all():
                if last_pos is None:
                    pass
                else:
                    diff = np.linalg.norm(last_pos - center)
                    print(diff)
                    cv.imshow("ball", ball)
                    cv.waitKey(0)
                ball_coord = (center, radius)
        vid_mask = cv.drawKeypoints(
            inv_mask,
            blobs,
            np.array([]),
            255,
            cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
        )
        last_pos = None
        if ball_coord:
            cv.circle(frame, ball_coord[0], ball_coord[1], (0, 0, 255), 2)
            last_pos = ball_coord[0]
        cv.imshow("Frame", frame)
        cv.waitKey(0)
        # vid_mask = cv.merge((mask_blobs, mask_blobs, mask_blobs))
        output.write(frame)
        
        keyboard = cv.waitKey(30)
        if keyboard == 'q' or keyboard == 27:
            break
        i += 1
    capture.release()
    output.release()

In [5]:
for video_num in range(0, 1):
    input_video_path = f"./imp/videos/videos_0/{video_num}.MOV"
    output_video_path = f"./imp/background_subtraction/{video_num}_ball_det.avi"
    back_sub_algo = "MOG2" # MOG2 or KNN
    background_subtraction(input_video_path, output_video_path, back_sub_algo)

0/500
10/500
20/500
26.0
24.166091947189145
30/500
22.847319317591726
21.93171219946131
20.591260281974
18.35755975068582
17.4928556845359
17.204650534085253
1681.071087134628
15.264337522473747
14.212670403551895
12.041594578792296
11.313708498984761
40/500
10.816653826391969
8.602325267042627
9.848857801796104
8.246211251235321
8.06225774829855
8.0
7.280109889280518
7.615773105863909
2270.9031243097975
8.06225774829855
10.0
50/500
9.899494936611665
2266.503915725715
10.816653826391969
12.206555615733702
13.038404810405298
13.92838827718412
15.652475842498529
16.15549442140351
17.08800749063506
18.681541692269406
2536.7327411455863
18.973665961010276
60/500
20.615528128088304
710.3590641358777
21.840329667841555
690.7626220345163
22.561028345356956
24.331050121192877
25.495097567963924
2235.2521110604057
2201.2616836714346
25.495097567963924
2163.1655045326515
2198.2340639704407
2025.33799648355
70/500
38.07886552931954
80/500
33.734255586866
90/500
33.0
35.0
100/500
40.31128874149274

Low: 45.27591707
High: 62.8883623